#Food Vision


## check GPU

The GPUs in the colab can be not compatible with the mixed precision training.

* K80 (not compatible)
* P100 (not compatible)
* Tesla T4 (compatible)

Inorder to use mixed precision training we need access to a Tesla T4 GPU.
or if using own GPU it needs a score of 7.0+

https://developer.nvidia.com/cuda-gpus

In [ ]:
!nvidia-smi -L

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Connect to TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU detected and initialized")
except:
    strategy = tf.distribute.get_strategy()  # Use default strategy if no TPU
    print("⚠️ TPU not found, using default strategy")


# Getting helper functions

In [ ]:
#using the helper funtions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
# Import helper functions that can be used
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

#using tensorflow datasets to download data

https://www.tensorflow.org/datasets/overview

In [ ]:
# Get Tensorflow Datasets
import tensorflow_datasets as tfds

In [ ]:
#List all available datasets
datasets_list = tfds.list_builders() #get all available datasets in TFDS
print("food101" in datasets_list) #is our target dataset in the list of TFDS dataset?


In [ ]:

(train_data, test_data), ds_info = tfds.load(name = "food101",
                                             split = ["train", "validation"],
                                             shuffle_files = True,
                                             as_supervised = True, #data gets returned in tuple format(data,label)
                                             with_info = True)

##Exploring the FOOD101 data from Tensorflow Datasets

we want to find:
* class names
* The shape of our inputs data (image tensor)
* The datatype of our input data
* Waht the labels look like(e.g are they one-hot encoded or are they label encoded)
* do the labels match up with the class names?

In [ ]:
# Feature of Food101 TFDS
ds_info.features

In [ ]:
#Get the class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
#Take one sample of the train data
train_one_sample = train_data.take(1) #samples are in format(image_tensor, label)
train_one_sample


In [ ]:
#output info about the training sample
for image, label in train_one_sample:
  print(f"""
  image shape: {image.shape}
  image datatype: {image.dtype}
  target class from Food101 (tensor form): {label}
  target class from label (integer): {label.numpy()}
  Class name (str form): {class_names[label.numpy()]}
  """)

In [ ]:
#what does the image tensor from the TFDS's FOOD101 look like
image

In [ ]:
# What are the min and max values of our image
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

###Plot an image from the tensorflow datasets


In [ ]:
#Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)

In [ ]:
plt.title(class_names[label.numpy()]) #Add title to the image to verify the label is assosciateed with the right image
plt.axis(False)

## Create preprocessing functions for our data
(Neural Network works best when data is in certain way (e.g. batched, normalized, etc).

not all data comes like this.

inorder to get it ready for a neural network, you'll often have to write preprocessing functions and map it to your data.

what we know about our data:
* In 'unit8' datatype
* Comprised of all different size tensors( different sized images)
* Not scaled (the pixel values are between 0 & 255)

what we know models like:
* Data in 'float32' dtype (or for mixed precision 'float16' and 'float32')
* For batches, TensorFlow likes all of the tensors within a batch to be of the same size
* Scaled (values between 0 & 1 ) also called normalized tensors generally perform better

we use EfficientNetBX pretrained model from tf.keras.applicatinos we don't need to architectures have rescaling built-in).

This means our funtions needs to:
1. Reshape our images to all the same size
2. Convert the dtype of our image tensors from uint8 to float32

In [ ]:
#(image,label)

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label ,img_shape = 224):
  """
  Converts image datatype from 'uint8' -> 'float32'
  Reshapes the image to [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape])# reshape target image
  image = image / 255.
  return tf.cast(image, tf.float32), label # return (float_image, label) tuple

In [ ]:
# Preprocess a single sample image and check the outputs
preprocessed_img = preprocess_img(image, label)[0]

print(f"Image before preprocessing:\n {image[:2]}...\n Shape: {image.shape},\nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}...,\n Shape: {preprocess_img.shape},\n Datatype :{preprocess_img.dtype}")
